# Numerical Model Code

The code for the numerical approximation model. In this notebook, as an example it's being run on the full network with no dams for the duration of storm Ciara. I make use of the @jit decorator from numba, which compiles code in order to allow it to run at speeds approaching a much faster language like C. This does not however support pandas, meaning a lot of the functions are defined by looping over numpy arrays. This is the efficient approach though, as a numerical IVP solver for this problem will need to evaluate the same functions many times, and compiling these functions will greatly reduce computational cost. <br> The imports and data needed:

In [1]:
# imports
import random
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from numba import jit
from datetime import datetime
from scipy.optimize import root
from scipy.integrate import solve_ivp
warnings.filterwarnings('ignore')
plt.style.use('seaborn-white')

# data - note terrain data is in a shapefile so need geopandas to unpack it
terrain = gpd.read_file('data/terrain/terrain.shp')
# set coordinate system
terrain.crs = {'init': 'espg3857'}

rain = pd.read_csv('data/edited_rain.csv', header=0, parse_dates=[0], index_col=0)  #inflow data
check = pd.read_csv('data/bowston_gauge_data.csv', header=0)  #Bowston gauge data, to check results against

widths = np.load(open('data/widths.npy','rb'))  #segment widths from checking-data.ipynb

# indexing correctly
check['datetime'] = pd.to_datetime(check['datetime'], dayfirst = True)
check = check.set_index('datetime')
rain.columns = [int(x)-1 for x in rain.columns]  #correcting column index

This code solves the following differential equations. For details see the writeup.

$$
\frac{d\mathbf{V}}{dt} = A^\mathsf{T}\mathbf{\tilde{Q}(\mathbf{h})} - \mathbf{\tilde{Q}(\mathbf{h})} + \mathbf{q}(t)
$$

Inputs:

In [2]:
pot_dams = np.load(open('data/pot_dams.npy','rb'))
# choose 100 dams at random
pot_ind = [i for i, x in enumerate(pot_dams) if x]

random.seed(0)  #makes random selection consistent through runs
dams = np.sort(random.sample(pot_ind, 100))
lower_heights = 0.3*np.ones(100)  #specifics of each dam. Must be same length as dams
upper_heights = 2*np.ones(100)
lambda_val = 10  #volume parameter, measures how much more volume each dam can store.
kvals = np.zeros(100)  #permeabilities - assumed to be 0 for each dam 


# timeframe of interest - this is for storm Ciara
start = pd.to_datetime('2020-02-08 6pm')
end = pd.to_datetime('2020-02-10 6pm')
eval_no = 100  #no. of points to evaluate solution at

General data handling:

In [3]:
N = terrain.shape[0]
Adj = np.zeros((N,N))  #adjacency matrix
for i in range(N):
    for j in range(N):
        if terrain['TONODE'][i] == terrain['FROMNODE'][j]:
            Adj[i,j]=1 

# getting the relevant data
data_df=terrain[['Shape_Leng','Slope']]
data_df.columns = ['l', 'S']
data_df['S'] = data_df['S'].apply(lambda x: x*-1)
data_df.loc[data_df.S<=0.1, 'S'] = 0.1

# set all dams to be large enough to have no impact on any realistic flow
data_df['w']=widths
data_df['h_l']=100*np.ones(N)  #h_l is lower dam height, h_u upper
data_df['h_u']=101*np.ones(N)
data_df['k']=np.zeros(N)

# change the required dam parameters to their correct values
for i, dam in enumerate(dams):
    data_df.k[dam] = kvals[i]
    data_df.h_l[dam] = lower_heights[i]
    data_df.h_u[dam] = upper_heights[i]

Making a function to get rain values at a time $t$ seconds after the start of the interval of interest. Note that the rain function is dimensional.

In [4]:
rain = rain[start:end]
rainvals = rain.to_numpy()
check = check[start:end]

@jit(nopython=True)
def rainfun(t):
    # redimensionalise time
    minutes = t/60
    # find number of 15 minute intervals that have passed:
    i = int((minutes - (minutes%15))/15)
    return rainvals[i,]

Nondimensionalising - see writeup for details:

In [5]:
# constants:
manning = 0.03  #typical manning constant
cst_w, cst_h, cst_S, cst_g, cst_l = 2., 1., 0.05, 9.8, 100.
cst_Q = cst_w*cst_h**(5/3)*cst_S**(1/2) / manning
cst_t = cst_l*cst_w*cst_h/cst_Q
cst_V = cst_l*cst_w*cst_h
cst_alpha = cst_h / cst_w
cst_gamma = np.sqrt(2*cst_g) * cst_h**(3/2) * cst_w / cst_Q
cst_beta = cst_h / (cst_S*cst_l)
cst_lambda = lambda_val*np.ones(N)

# putting everything into a central normalised dataframe:
df = pd.DataFrame()
df['l'] = data_df['l']/cst_l
df['S'] = data_df['S']/cst_S
df['w'] = data_df['w']/cst_w
df['h_l'] = data_df['h_l']/cst_h
df['h_u'] = data_df['h_u']/cst_h
df['k'] = data_df['k']  #k is a nondimensional constant
df = df.apply(pd.to_numeric, errors='coerce')

#immediately leave pandas for compatibility with numba.jit :(
l = tuple(df.l.tolist())
S = tuple(df.S.tolist())
w = tuple(df.w.tolist())
h_l = tuple(df.h_l.tolist())
h_u = tuple(df.h_u.tolist())
k = tuple(df.k.tolist())

Defining the normalised flow function $\mathbf{\tilde{Q}(\mathbf{h})}$. First, the critical values $h_c$ have to be calculated:

In [6]:
# define Qfric and Qdam
def Qfric(h, i):
    Qf = w[i]*h**(5/3) * np.sqrt(S[i])
    return Qf

def Qdam(h, i):
    a1 = h_l[i]*h / np.sqrt(h+h_l[i])
    a2 = k[i] * np.sqrt(h) * min( h-h_l[i], h_u[i]-h_l[i] )
    a3 = 0.6*2/3 * max(0, h-h_u[i])**(3/2)
    Qd = cst_gamma * w[i] * (a1+a2+a3)
    return  Qd


# find hcrit values
hcrit=[]
for i in range(N):
    def fun(z):
        fric = Qfric(z, i)
        dam = Qdam(z, i)
        return fric-dam
    hcrit.append(root(fun, h_u[i]).x[0])
hcrit = tuple(hcrit)  #make immutable

Defining $\mathbf{\tilde{Q}(\mathbf{h})}$ and $\mathbf{\tilde{V}(\mathbf{h})}$ properly. Note these are not vector valued functions, instead are a family of functions indexed by segment number $i$. See writeup for explanation of functions used.

In [7]:
def Qfun(h, i):
    if h<max(hcrit[i], h_l[i]):
        return  Qfric(h, i)
    else:
        return Qdam(h, i)
    
    
def Vfun(h, i):
    hstr = (Qfun(h,i) /w[i] /np.sqrt(S[i]))**(3/5)  #hstream downstream flow
    if h < hstr + S[i]*l[i]/cst_beta:
        return w[i]*l[i]*hstr + cst_lambda[i]*0.5*cst_beta*w[i]* max(0, h-hstr)**2 / S[i]
    else:
        return w[i]*l[i]*( h - S[i]*l[i]/cst_beta + cst_lambda[i]/2*S[i]*l[i]/cst_beta)
     

Define an approximation for $\mathbf{\tilde{Q}(\mathbf{V})}$ - note $\mathbf{\tilde{V}(\mathbf{h})}$ is made to be monotonic, this comes from having to invert it. This corresponds to  $min\circ\mathbf{\tilde{V}^{-1}}$, as the inverse may be a multifunction.

In [8]:
# approximate V(h)
Vapprox = np.zeros((N, 100))
for i in range(N):
    runmax=0  #for monotonicity
    for j, h in enumerate(np.linspace(0, 20, 100)):
        runmax = max(runmax, Vfun(h, i))
        Vapprox[i, j] = runmax
        
# approximate Q(h)
Qapprox = np.zeros((N,100))
for i in range(N):
    for j, h in enumerate(np.linspace(0, 20, 100)):
        Qapprox[i,j] = Qfun(h,i)

# interpolate using the above approximations
@jit(nopython=True)
def Qvals(V):
    Q = np.zeros((N,1))
    for i in range(N):
        Q[i,0] = np.interp(V[i], Vapprox[i,:], Qapprox[i,:])
    return Q

Define the derivative $D$ at a point, using the above function to find $\mathbf{Q}$ given $\mathbf{V}$.

In [9]:
@jit(nopython=True)
def derivative(t, V):
    #inflow
    q = np.zeros((N,1))
    q[:,0] = rainfun(t*cst_t)/cst_Q
    #working out Q:
    Q = Qvals(V)
    #dV/dt
    D = Adj.T.dot(Q) - Q  + q
    return list(D.flat)

Since this is not a stiff problem, I'm using the standard IVP solver, with a Runge-Kutta method.

In [10]:
initial = [0]*(N)
# derivative2 will only use the t=0 inflow to make a suitable baseflow.
def derivative2(t, V):
    return derivative(0, V)
sol = solve_ivp(derivative2, (0,60*60/cst_t), initial, atol=1e-3)
initial = sol.y[:,-1]


T = (end-start).total_seconds() / cst_t
startTime = datetime.now()
sol = solve_ivp(derivative, (0,T), initial, t_eval=np.linspace(0,T,100) , atol=1e-3, method='RK23')
print('The total time taken is ' + str((datetime.now()-startTime).total_seconds())+' seconds.')

The total time taken is 625.529208 seconds.


The most useful output are flow values. These can be calculated from Qfun defined above. I'm saving the flow values to a csv.

In [11]:
# for this, populate an empty dataframe of the right shape

def qfunexp(V, i):
    return np.interp(V, Vapprox[i,:], Qapprox[i,:])

qoutput = pd.DataFrame(sol.y.copy().T, index=pd.date_range(start, end, 100))
for i, col in enumerate(qoutput.columns):
    qoutput[col] = qoutput[col].apply(lambda x: qfunexp(x, i)*cst_Q)
qoutput.to_csv('data/numerical_flow_data.csv')

Additionally, I'll output data about how filled each dam is - this is to help visualising the solution. Here I'll calculate how high the segment is in comparison to the impacting dam surface.

In [12]:
dam_df = pd.DataFrame(sol.y[dams].copy().T, index = pd.date_range(start, end, 100), columns=dams)
for dam in dam_df.columns:
    dam_df[dam] = dam_df[dam].apply(lambda x: (x>df.h_l[dam]) * min(1, (x-df.h_l[dam])/(df.h_u[dam]-df.h_l[dam]) ))
dam_df.to_csv('data/numerical_dam_data.csv')